In [9]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn import set_config
from time import time
from datetime import datetime
from dateutil import parser
from sklearn.model_selection import cross_validate
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import SVC
import random

In [3]:
def prep_df(csv):
    global df_original
    df_original = pd.read_csv(csv)
    df = pd.read_csv(csv)
    df["date_today"] = datetime.now()
    df = df.dropna(subset = ["original_estimate_complete_datetime"])
    df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
    df = df.assign(time_since_estimate =  df['date_today'] - df['estimate_date_parse'])
    df["days_since_estimate"] = df["time_since_estimate"].apply(lambda x: int(str(x).split()[0]))
   
    del df["time_since_estimate"]
    df1 = df[df["suppCnt"] == 0]
    df2 = df[df["suppCnt"] != 0]
    df1["suppInd"] = 0
    df2["suppInd"] = 1
    df = pd.concat([df1,df2], axis = 0)
    df["paid_loss_amount"] = df["paid_loss_amount"].apply(lambda x: abs(x))
    df["primary_point_of_impact"].fillna("Unknown",inplace = True)
    df["total_loss_points"].fillna(999,inplace = True)
    df["insured_fault_pct"].fillna(999,inplace = True)
    global df_w_days
    df_w_days = df
    df = df[["claim_number","company","coverage_grouping","detailed_loss_cause",'driveable_ind',"insured_fault_pct","insured_fault_rating","line_of_business","loss_cause","loss_party_type","loss_state","original_estimate_complete_datetime","payAfterCloseInd","policy_state","policy_type","primary_point_of_impact","saved_vehicle_as_type","secondary_point_of_impact","total_loss_points","vehicle_make","vehicle_year","suppInd","maxNet","minNet","maxFile","minFile","days_since_estimate"]]
    global df_full_data
    df_full_data = df
    df = df[df["days_since_estimate"] >= 90]
    return df

In [4]:
time1 = time()
df = prep_df("Supp Prediction.csv")

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_18572\1398384070.py:3: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_18572\1398384070.py:4: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_18572\1398384070.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_18572\1398384070.py:14: SettingWithCopyWarning: 
A value is trying to be set on 

In [13]:
randomlist = []
randomlist = random.sample(range(0, (len(df) - 1)), 100000)
df_sample = df.iloc[randomlist]
df_sample

,claim_number,company,coverage_grouping,detailed_loss_cause,driveable_ind,insured_fault_pct,insured_fault_rating,line_of_business,loss_cause,loss_party_type,...,secondary_point_of_impact,total_loss_points,vehicle_make,vehicle_year,suppInd,maxNet,minNet,maxFile,minFile,days_since_estimate
240562,01-003-997734,MSA,Property Damage Liability,Sideswipe,No,90.0,Insured Majority at Fault,Commercial Auto Line,Collision with Motor Vehicle/Watercraft,Third-party liability,...,NaN,29.0,Nissan,2017,0,14180.04,14180.04,14180.04,14180.04,289
186384,01-004-557532,Connect,Collision,Hit and Run,Yes,0.0,Claimant Majority at Fault,Personal Auto Line,Collision with Stationary/Other Object,Insured's loss,...,NaN,28.0,Acura,2009,1,1461.29,1277.14,1711.29,1527.14,118
132125,01-003-744475,Connect,Collision,Claimant Rearended Insured,Yes,0.0,Claimant Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,NaN,9.0,Chevrolet Truck,2019,1,12322.58,11274.79,12822.58,11774.79,323
138172,01-004-353694,Amfam,Property Damage Liability,Backing/Reversing,Yes,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Third-party liability,...,NaN,15.0,Hyundai,2016,0,805.38,805.38,805.38,805.38,119
172759,01-003-954856,Amfam,Collision,Claimant Rearended Insured,Yes,0.0,Claimant Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,NaN,14.0,Chevrolet Truck,2015,1,6954.48,6451.42,7530.45,7009.45,302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130170,01-004-523611,MSA,Collision,T-bone,No,0.0,Claimant Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,NaN,26.0,Chevrolet Truck,2018,1,8064.42,6406.08,8564.42,6906.08,135
167076,01-004-542886,Connect,Collision,Front End Collision,Yes,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Stationary/Other Object,Insured's loss,...,NaN,25.0,Hyundai,2015,1,3558.20,1451.18,4058.20,1951.18,132
92169,01-003-924865,Amfam,Collision,Head On Collision,Yes,0.0,Claimant Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,NaN,14.0,Chevrolet Truck,2016,0,1246.36,1246.36,1746.36,1746.36,260
91875,01-003-898728,Amfam,Collision,Collision While Turning,No,100.0,Insured Majority at Fault,Personal Auto Line,Collision with Motor Vehicle/Watercraft,Insured's loss,...,NaN,11.0,Gmc Truck,2017,1,7387.74,7020.75,8387.74,8020.75,320


In [14]:
df_sample

target_name = "suppInd"
Y = df_sample[target_name]
X = df_sample.drop(columns=[target_name,"payAfterCloseInd","days_since_estimate"])

In [17]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])

In [18]:
model = make_pipeline(preprocessor, SVC(kernel='linear', cache_size = 7000))

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

In [ ]:
svc = model.fit(X_train, Y_train)


In [ ]:
svc.score(X_test, Y_test)

In [ ]:
df_under_90 = df_full_data[df_full_data["days_since_estimate"] < 90]
df_under_90["prediction"] = clf.predict(df_under_90)
time2 = time()

In [ ]:
len(df_under_90[df_under_90["prediction"] == 1]) / len(df_under_90)

In [ ]:
len(df[df["suppInd"] == 1]) / len(df)